In [1]:
import discord
from discord.ext import tasks, commands
from datetime import datetime
import requests
import bs4
import nest_asyncio
nest_asyncio.apply()

file = open("token.txt", mode="r")
token = file.readline().strip()

In [2]:
def coin_price(url):
    req = requests.get(url)
    soup = bs4.BeautifulSoup(req.text)
    e = str(soup.find_all(attrs={'class': 'price'})[0])
    price = float(e[e.index("-->")+3:e.index("</")])
    
    return price

In [3]:
prev_link_price = 0.01
prev_flow_price = 0.01
client = commands.Bot(command_prefix = ".")

@client.event
async def on_ready():
    print_prices.start()
    print("Bot initialized")
    
@tasks.loop(seconds=3600)
async def print_prices():
    global prev_link_price
    global prev_flow_price
    
    channel = client.get_channel(745923825727963166)
    
    link_price = coin_price("https://www.livecoinwatch.com/price/ChainLink-LINK")
    flow_price = coin_price("https://www.livecoinwatch.com/price/FlowProtocol-FLOW")
    
    message = "**---------------\nHOURLY UPDATE: *" + datetime.now().strftime("%H:%M:%S") + "***\n```LINK: \n$" + str(link_price) + " (" + str(100*(link_price-prev_link_price)/prev_link_price) + "%) "
    message += ("🚀" if link_price > prev_link_price else "📉" if link_price < prev_link_price else "🤔") + "\n\n"
    
    message += "FLOW: \n$" + str(flow_price) + " (" + str(100*(flow_price-prev_flow_price)/prev_flow_price) + "%) "
    message += ("🚀" if flow_price > prev_flow_price else "📉" if flow_price < prev_flow_price else "🤔") + "```"
    
    await channel.send(message)
    prev_link_price = link_price
    prev_flow_price = flow_price
        
client.run(token)

Bot initialized


RuntimeError: Cannot close a running event loop